Import necessary packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

Import datasets and packages

In [38]:
df_lookup = pd.read_csv('../../data/raw/EXTR_LookUp.csv', dtype='str')
#df_parcel = pd.read_csv('../../data/raw/EXTR_PARCEL.csv', dtype='str')
df_resbldg = pd.read_csv('../../data/raw/EXTR_ResBldg.csv', dtype='str')
df_rpsale = pd.read_csv('../../data/raw/EXTR_RPSale.csv', dtype='str')

Strip leading and trailing spaces

In [39]:
from custom_functions import strip_spaces

df_lookup = strip_spaces(df_lookup)
#df_parcel = strip_spaces(df_parcel)
df_resbldg = strip_spaces(df_resbldg)
df_rpsale = strip_spaces(df_rpsale)

In [40]:
df_resbldg

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,DirectionSuffix,ZipCode,Stories,BldgGrade,BldgGradeVar,SqFt1stFloor,SqFtHalfFloor,SqFt2ndFloor,SqFtUpperFloor,SqFtUnfinFull,SqFtUnfinHalf,SqFtTotLiving,SqFtTotBasement,SqFtFinBasement,FinBasementGrade,SqFtGarageBasement,SqFtGarageAttached,DaylightBasement,SqFtOpenPorch,SqFtEnclosedPorch,SqFtDeck,HeatSystem,HeatSource,BrickStone,ViewUtilization,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,009800,0720,1,1,27719 SE 26TH WAY 98075,27719,,SE,26TH,WAY,,98075,2,11,0,1970,0,2130,0,0,0,4100,0,0,0,0,750,,0,0,0,5,2,0,,4,1,0,3,3,0,0,0,2001,0,0,0,0,3,0
1,009802,0140,1,1,2829 277TH TER SE 98075,2829,,,277TH,TER,SE,98075,2,10,0,1610,0,1400,0,0,0,3010,0,0,0,0,660,,380,0,0,5,2,0,,4,1,2,2,3,0,0,0,2004,0,0,0,0,3,0
2,009830,0020,1,1,1715 298TH CRESENT SE,1715,,,298TH CRESENT,,SE,NaN,2,10,0,2520,0,2560,0,0,0,5080,0,0,0,0,1020,,360,0,270,5,2,0,,4,0,0,5,1,0,0,0,2017,0,0,0,0,3,0
3,009830,0160,1,1,1861 297TH WAY SE 98024,1861,,,297TH,WAY,SE,98024,2,10,0,2210,0,1860,0,0,0,4070,0,0,0,0,1000,N,690,0,0,5,2,0,N,4,1,0,3,2,0,0,0,2013,0,0,0,0,3,0
4,010050,0180,1,1,35410 25TH PL S 98003,35410,,,25TH,PL,S,98003,2,7,0,910,0,700,0,0,0,1610,0,0,0,0,440,,60,0,0,5,2,0,,3,1,1,1,1,0,0,0,1994,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181505,197220,1902,1,1,3816 WINSLOW PL N 98103,3816,,,WINSLOW,PL,N,98103,3,8,0,150,0,450,430,0,0,1030,0,0,0,0,240,,0,0,0,5,2,0,,2,1,0,1,1,0,0,0,2003,0,0,0,0,3,0
181506,197220,2224,1,1,3841 B WOODLAND PARK AVE N 98103,3841,B,,WOODLAND PARK,AVE,N,98103,3,8,0,260,0,580,590,0,0,1430,0,0,0,0,270,N,0,0,50,5,2,0,Y,3,1,0,2,1,0,0,0,2005,0,0,0,0,3,0
181507,197220,5172,1,1,11 B W DRAVUS ST,11,B,W,DRAVUS,ST,,NaN,3,8,0,220,0,510,380,0,0,1600,490,490,8,0,290,,0,0,0,5,2,0,,3,1,0,2,0,0,0,0,2007,0,0,0,0,3,0
181508,197220,5173,1,1,11 C W DRAVUS ST,11,C,W,DRAVUS,ST,,NaN,3,8,0,200,0,490,380,0,0,1470,400,400,8,0,230,,0,0,70,5,2,0,,3,1,0,2,0,0,0,0,2007,0,0,0,0,3,0


## Refine *Sales* DataFrame

In [41]:
res_sales = df_rpsale.copy()

principal_use_codes_to_keep = ['6'] # Consider investigating '4' too 
principal_use_codes_to_drop = np.setdiff1d(res_sales.PrincipalUse.unique(), principal_use_codes_to_keep)

property_class_codes_to_keep = ['8']
property_class_codes_to_drop = np.setdiff1d(res_sales.PropertyClass.unique(), property_class_codes_to_keep)

# Need to consider where to classify codes like 2,3 since there isn't an explicit statement
#    - Possible to disregard entirely, explore data first
# Consider looking into single-family and multiple-family separately
property_type_codes_to_keep =['11']# ['2', '3', '6', '10', '11', '12', '13', '18', '19']
property_type_codes_to_drop = np.setdiff1d(res_sales.PropertyType.unique(), property_type_codes_to_keep)

for code in principal_use_codes_to_drop:
    res_sales['PrincipalUse'].replace(to_replace=code, value=np.nan, inplace=True)

for code in property_class_codes_to_drop:
    res_sales['PropertyClass'].replace(to_replace=code, value=np.nan, inplace=True)

for code in property_type_codes_to_drop:
    res_sales['PropertyType'].replace(to_replace=code, value=np.nan, inplace=True)

# CREATE PARCEL ID
res_sales['Parcel_ID'] = res_sales.Major + '-' + res_sales.Minor



# KEEP ONLY 2019 SALES
res_sales['DocumentDate'] = res_sales.DocumentDate.astype(np.datetime64)
res_sales['SaleYear'] = [sale.year for sale in res_sales['DocumentDate']]
res_sales = res_sales.loc[res_sales['SaleYear']==2019].copy()



# ELIMINATE SALES OF ZERO DOLLARS
res_sales['SalePrice'] = res_sales.SalePrice.astype('int')
res_sales['SalePrice'].replace(0, np.nan, inplace=True)

# DROP SALES DETERMINED TO BE INVALID
res_sales.dropna(inplace=True)


# CREATE COLUMN TO IDENTIFY DUPLICATES WHEN MAPPING TO RESBLDG DATAFRAME
res_sales['SaleCount'] = list(map(dict(res_sales.Parcel_ID.value_counts()).get, res_sales.Parcel_ID))
res_sales.head(3)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,Parcel_ID,SaleYear,SaleCount
9,3027422,213043,0120,2019-12-20,560000.0,20191226000848,,,,,,,DOYLE REGAN M+STERLING C,SHAW HEATHER C,11,6,3,N,N,N,N,1,8,,213043-0120,2019,1
10,3002257,940652,0630,2019-07-22,435000.0,20190730001339,,,,,,,CRAMER NEIL C+PAM R+ET AL,HANSON BRYAN L+KAILI,11,6,3,N,N,N,N,1,8,,940652-0630,2019,1
140,2982060,347050,0040,2019-04-04,648500.0,,,,,,,,BROWN MARK E+MARCIE K,MEDVED ROBERT J+JENNIFER E,11,6,3,N,N,N,N,1,8,,347050-0040,2019,1


**Join the two main tables on Parcel_ID**

In [43]:
res_sales.loc[res_sales.SaleCount>1].sort_values(by='Parcel_ID')

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,Parcel_ID,SaleYear,SaleCount
11442,3017832,001600,0140,2019-10-25,345000.0,,,,,,,,KNITTLE JILL CHARDON+DOUGLAS E,HAVENS JACOB L+TOVES HALEY C,11,6,3,N,N,N,N,1,8,,001600-0140,2019,2
227036,2973078,001600,0140,2019-01-28,295000.0,20190211000209,,,,,,,CURRER JEREMY,KNITTLE JILL CHARDON+DOUGLAS E,11,6,3,N,N,N,N,1,8,,001600-0140,2019,2
309847,2986122,004100,0250,2019-04-27,323677.0,20190503001352,,,,,,,SNIDER GREGORY E+CHRISTINA N+ET AL,ROUSH MELANIE+DOUGLAS,11,6,3,N,N,N,N,1,8,,004100-0250,2019,2
244945,3026786,004100,0250,2019-12-05,366000.0,20191220001804,,,,,,,EDWARDS DARON M.,HAMPTON SHELDON LEE II,11,6,3,N,N,N,N,1,8,,004100-0250,2019,2
36195,3020449,005330,0370,2019-10-23,685000.0,20191114000129,,,,,,,ALLEN HEATHER L+CHRISTOPHER R,TRC GLOBAL MOBILITY INC,11,6,3,N,N,N,N,1,8,40,005330-0370,2019,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92853,2972655,955805,0190,2019-01-20,784500.0,20190206001487,,,,,,,SIRVA RELOCATION CREDIT LLC,BROWN HUGH J+REBECCA S+ET AL,11,6,3,N,N,N,N,1,8,41,955805-0190,2019,2
276292,2995042,955820,0210,2019-06-10,425000.0,20190621000744,,,,,,,NUCOMPASS MOBILITY SERVICES INC,LIBBEY JOHN PATRICK,11,6,3,N,N,N,N,1,8,41,955820-0210,2019,2
144718,2995043,955820,0210,2019-06-10,425000.0,20190621000747,,,,,,,OVERALL DAVID,NUCOMPASS MOBILITY SERVICES INC,11,6,3,N,N,N,N,1,8,40,955820-0210,2019,2
185969,3017516,957801,0220,2019-10-15,709950.0,20191028000877,,,,,,,NATIONAL RESIDENTIAL NOMINEE SERVICES INC,BANDYOPADHYAY ARITRA+CHIROSREE,11,6,3,N,N,N,N,1,8,41,957801-0220,2019,2


In [19]:
old_res_index = res_sales.index
old_porch_index = porch.index    

In [47]:
#res_sales.reset_index(drop=True, inplace=True)
res_sales.loc[res_sales.SaleCount==6].sort_values(by='Parcel_ID')

#porch.index = old_porch_index

#porch.loc[porch.Parcel_ID == '312206-9048']
#porch.loc[porch.Parcel_ID == '082211-9001']

#porch['ParcelCount'] = 
#porch['ParcelCount'] = list(map(dict(porch.Parcel_ID.value_counts()).get, porch.Parcel_ID))
#porch

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,Parcel_ID,SaleYear,SaleCount
41899,2975737,884390,0445,2019-02-14,815000.0,20190228001472,,,,,,,1404-WLD ISSAQUAH TOWNHOMES LLC,SCOTT BRYAN T+NICOLE,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
149957,2975141,884390,0445,2019-02-14,699900.0,20190226001189,,,,,,,1404 WLD ISSAQUAH TOWNHOMES LLC,FRATES ANDREA J+KLEIN ALEXANDER WK,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
197942,2976965,884390,0445,2019-03-01,706900.0,20190311000874,,,,,,,1404 WLD ISSAQUAH TOWNHOMES LLC,GOPALAN YADHU,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
292126,2980974,884390,0445,2019-03-21,799900.0,20190405000497,,,,,,,1404 WLD ISSAQUAH TOWNHOMES LLC,DIETZEL GREGORY B+CHERYL A,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
294166,2975973,884390,0445,2019-02-15,782900.0,20190301000951,,,,,,,1404-WLD ISSAUQUAH TOWNHOMES LLC,LINGG JASON+BURROW MICHELLE,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
312455,2975726,884390,0445,2019-02-14,776900.0,20190228001436,,,,,,,1404-WLD ISSQUAH TOWNHOMES LLC,GRIFFIN SUSAN C,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6


In [156]:
def identify_latest_sale(docdates, parcel_ids):
    latest_parcel_sale = []
    data = pd.DataFrame([docdates, parcel_ids]).T
    data.DocumentDate = data.DocumentDate.astype('datetime64')
 
    for i, parcel_id in enumerate(data.Parcel_ID):
        #if i>100:
        #    break
        relevant_docdates = data.loc[data.Parcel_ID == parcel_id, 'DocumentDate']
        max_docdate = relevant_docdates.values.max()
        
        this_datetime = np.datetime64(data.iloc[i, 0]) 
        latest_parcel_sale.append(this_datetime == max_docdate)

    return latest_parcel_sale

        
        
dd = res_sales.loc[res_sales.SaleCount > 1, 'DocumentDate']
pi = res_sales.loc[res_sales.SaleCount > 1 , 'Parcel_ID']
tf = identify_latest_sale(dd, pi)
res_sales.loc[res_sales.SaleCount > 1].loc[tf]

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,Parcel_ID,SaleYear,SaleCount
192,3020690,345960,0230,2019-10-30,1255000.0,20191114001849,,,,,,,EEP JIAGEN+PEH WEI PING,GRAEBEL RELOCATION SERVICES WORLDWIDE INC,11,6,3,N,N,N,N,1,8,40,345960-0230,2019,2
429,3016016,941461,0060,2019-10-04,810000.0,20191018000776,,,,,,,ENGARD DEREK+NICHELLE,RELO DIRECT GOVERNMENT SERVICES LLC,11,6,3,N,N,N,N,1,8,40,941461-0060,2019,2
1230,3027185,261010,0045,2019-12-16,610000.0,20191224000206,,,,,,,LA INVESTMENTS LLC,HEENEY TIMOTHY E,11,6,3,N,N,N,N,1,8,,261010-0045,2019,2
1759,3016068,276371,0030,2019-10-08,810000.0,20191018001141,,,,,,,LOVELESS CHRIS+TANAKA HITOMI,BALEST ADAM M+GOLDBERG LEIGH B,11,6,3,N,N,N,N,1,8,,276371-0030,2019,2
3469,3014137,333990,1140,2019-10-01,349000.0,20191007001535,,,,,,,AMERICAN INTERNATIONAL RELOCATION SOLUTIONS LLC,HORETH DANIEL+LAPLANTE MEGAN,11,6,3,N,N,N,N,1,8,41,333990-1140,2019,2
3890,3038346,312206,9048,2019-08-27,4704.0,,,,,,,,ALQUIST CHRIS,COVINGTON CITY OF,11,6,26,N,N,N,N,18,8,,312206-9048,2019,4
4739,3011098,201320,0170,2019-09-12,420000.0,20190919001648,,,,,,,SCHWARTZ SAM,CHAVEZ SIMON,11,6,3,N,N,N,N,1,8,,201320-0170,2019,2
5382,2975652,019110,0770,2019-02-14,1308550.0,20190228001200,,,,,,,SIRVA RELOCATION CREDIT LLC,GRUTZMACHER RICHARD P III+MORLEY REBECCA M,11,6,3,N,N,N,N,1,8,41,019110-0770,2019,2
5742,3025506,010050,0380,2019-12-12,445000.0,20191213001173,,,,,,,VOVK CONSTRUCTION INC.,CENIDO AZHJELI,11,6,3,N,N,N,N,1,8,,010050-0380,2019,2
9307,3011617,020790,0010,2019-09-13,899000.0,20190923002220,,,,,,,HARLOW WILLIAM F III+KATHRYN LEE,BGRS LLC,11,6,3,N,N,N,N,1,8,40,020790-0010,2019,2


In [128]:
res_sales.loc[res_sales.SaleCount==6]

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,Parcel_ID,SaleYear,SaleCount
41899,2975737,884390,0445,2019-02-14,815000.0,20190228001472,,,,,,,1404-WLD ISSAQUAH TOWNHOMES LLC,SCOTT BRYAN T+NICOLE,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
149957,2975141,884390,0445,2019-02-14,699900.0,20190226001189,,,,,,,1404 WLD ISSAQUAH TOWNHOMES LLC,FRATES ANDREA J+KLEIN ALEXANDER WK,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
197942,2976965,884390,0445,2019-03-01,706900.0,20190311000874,,,,,,,1404 WLD ISSAQUAH TOWNHOMES LLC,GOPALAN YADHU,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
292126,2980974,884390,0445,2019-03-21,799900.0,20190405000497,,,,,,,1404 WLD ISSAQUAH TOWNHOMES LLC,DIETZEL GREGORY B+CHERYL A,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
294166,2975973,884390,0445,2019-02-15,782900.0,20190301000951,,,,,,,1404-WLD ISSAUQUAH TOWNHOMES LLC,LINGG JASON+BURROW MICHELLE,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6
312455,2975726,884390,0445,2019-02-14,776900.0,20190228001436,,,,,,,1404-WLD ISSQUAH TOWNHOMES LLC,GRIFFIN SUSAN C,11,6,3,N,N,N,N,1,8,,884390-0445,2019,6


In [61]:
temp = pd.DataFrame([res_sales.DocumentDate, res_sales.Parcel_ID]).T

temp.DocumentDate = temp.DocumentDate.astype('datetime64')
temp

,DocumentDate,Parcel_ID
9,2019-12-20,213043-0120
10,2019-07-22,940652-0630
140,2019-04-04,347050-0040
192,2019-10-30,345960-0230
208,2019-03-04,030200-0645
...,...,...
351029,2019-09-16,374670-0090
351036,2019-05-21,405940-0095
351037,2019-07-22,381670-0025
351038,2019-11-18,615020-0505


In [48]:
#deduped_res_sales = 

#res_sales.loc[res_sales.Parcel_ID.value_counts()>1]
#res_sales.index = res_sales.Parcel_ID
#porch.index = porch.Parcel_ID
#pd.concat([porch, res_sales], axis=1, join='inner')

In [9]:
#res_sales.loc[res_sales.Parcel_ID == '532310-0141']
#res_sales.loc[res_sales.PropertyType == '11']




In [10]:
df_resbldg.loc[(df_resbldg.Major=='532310')]# & (df_resbldg.Minor=='0141')]

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,DirectionSuffix,ZipCode,Stories,BldgGrade,BldgGradeVar,SqFt1stFloor,SqFtHalfFloor,SqFt2ndFloor,SqFtUpperFloor,SqFtUnfinFull,SqFtUnfinHalf,SqFtTotLiving,SqFtTotBasement,SqFtFinBasement,FinBasementGrade,SqFtGarageBasement,SqFtGarageAttached,DaylightBasement,SqFtOpenPorch,SqFtEnclosedPorch,SqFtDeck,HeatSystem,HeatSource,BrickStone,ViewUtilization,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
66570,532310,0115,1,1,3032 61ST AVE SW,3032,,,61ST,AVE,SW,NaN,2,8,0,630,0,630,0,0,0,1700,620,440,8,180,0,Y,0,0,110,3,2,0,N,3,1,0,3,1,0,0,0,2010,0,0,0,0,3,0
66599,532310,0110,1,1,3024 A 61ST AVE SW,3024,A,,61ST,AVE,SW,NaN,2,9,0,630,0,580,0,0,0,1780,570,570,8,0,0,Y,60,100,530,5,2,0,N,3,1,1,1,0,0,0,0,2016,0,0,0,0,3,0
66600,532310,0134,1,1,3042 B 61ST AVE SW 98116,3042,B,,61ST,AVE,SW,98116,2,8,0,400,0,370,0,0,0,1040,270,270,7,0,0,Y,60,0,0,5,2,0,N,2,0,1,1,1,0,0,0,2006,0,0,0,0,3,0
66609,532310,0130,1,1,3044 61ST AVE SW 98116,3044,,,61ST,AVE,SW,98116,1,7,0,890,0,0,0,0,0,1780,890,890,7,0,0,N,0,0,0,5,2,0,N,3,0,0,2,1,0,0,0,1941,0,0,0,0,4,0
66629,532310,0122,1,1,3036 61ST AVE SW,3036,,,61ST,AVE,SW,NaN,2,8,0,560,0,560,0,0,0,1360,540,240,7,300,0,Y,0,0,100,3,2,0,,3,1,0,2,1,0,0,0,2010,0,0,0,0,3,0
66807,532310,0112,1,1,3024 C 61ST AVE SW,3024,C,,61ST,AVE,SW,NaN,2,9,0,610,0,610,0,0,0,1620,400,400,8,0,0,Y,0,100,510,5,2,0,,3,1,1,1,0,0,0,0,2016,0,0,0,0,3,0
67054,532310,0132,1,1,3042 A 61ST AVE SW 98116,3042,A,,61ST,AVE,SW,98116,2,8,0,400,0,370,0,0,0,970,310,200,7,110,0,Y,60,0,0,5,2,0,N,2,0,1,1,1,0,0,0,2006,0,0,0,0,3,0
84734,532310,0111,1,1,3024 B 61ST AVE SW,3024,B,,61ST,AVE,SW,NaN,2,9,0,630,0,580,0,0,0,1780,570,570,8,0,0,Y,60,100,530,5,2,0,,3,1,1,1,0,0,0,0,2016,0,0,0,0,3,0
84939,532310,0117,1,1,3030 61ST AVE SW,3030,,,61ST,AVE,SW,NaN,2,8,0,560,0,560,0,0,0,1360,540,240,8,300,0,Y,0,0,100,3,2,0,N,3,1,0,2,1,0,0,0,2010,0,0,0,0,3,0


**Create DataFrame for Porch metrics**

In [11]:
porch = df_resbldg[['SqFtOpenPorch', 'SqFtEnclosedPorch']].copy()
porch['Parcel_ID'] = df_resbldg['Major'] + '-' + df_resbldg['Minor']
porch = porch[['Parcel_ID', 'SqFtOpenPorch', 'SqFtEnclosedPorch']].copy()

porch['SqFtOpenPorch'] = porch['SqFtOpenPorch'].astype('int')
porch['SqFtEnclosedPorch'] = porch['SqFtEnclosedPorch'].astype('int')
porch

,Parcel_ID,SqFtOpenPorch,SqFtEnclosedPorch
0,009800-0720,0,0
1,009802-0140,380,0
2,009830-0020,360,0
3,009830-0160,690,0
4,010050-0180,60,0
...,...,...,...
181505,197220-1902,0,0
181506,197220-2224,0,0
181507,197220-5172,0,0
181508,197220-5173,0,0


**Create DataFrame for Heating metrics**

In [12]:
from custom_functions import one_hot
from custom_functions import get_lookups

heating = one_hot(df_resbldg['HeatSystem'], name_lookup = get_lookups(108, df_lookup))
heating['binary_notforced'] = heating.drop('Forced Air', axis=1).sum(axis=1)
heating['binary_forced'] = heating['Forced Air']

**Bedroom and garage specs**

In [13]:
garage = df_resbldg[['SqFtGarageAttached', 'SqFtGarageBasement']].astype('int')
bedrooms = df_resbldg['Bedrooms'].astype('int')

**Aggregate Porch, Heating and Bedroom/Garage metrics**

In [14]:
porch_heating_rooms = pd.concat([porch, heating, garage, bedrooms], axis=1)
porch_heating_rooms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181510 entries, 0 to 181509
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Parcel_ID           181510 non-null  object
 1   SqFtOpenPorch       181510 non-null  int32 
 2   SqFtEnclosedPorch   181510 non-null  int32 
 3   Floor-Wall          181510 non-null  int32 
 4   Gravity             181510 non-null  int32 
 5   Radiant             181510 non-null  int32 
 6   Elec BB             181510 non-null  int32 
 7   Forced Air          181510 non-null  int32 
 8   Hot Water           181510 non-null  int32 
 9   Heat Pump           181510 non-null  int32 
 10  Other               181510 non-null  int32 
 11  binary_notforced    181510 non-null  int64 
 12  binary_forced       181510 non-null  int32 
 13  SqFtGarageAttached  181510 non-null  int32 
 14  SqFtGarageBasement  181510 non-null  int32 
 15  Bedrooms            181510 non-null  int32 
dtypes:

In [15]:
porch_heating_rooms

,Parcel_ID,SqFtOpenPorch,SqFtEnclosedPorch,Floor-Wall,Gravity,Radiant,Elec BB,Forced Air,Hot Water,Heat Pump,Other,binary_notforced,binary_forced,SqFtGarageAttached,SqFtGarageBasement,Bedrooms
0,009800-0720,0,0,0,0,0,0,1,0,0,0,0,1,750,0,4
1,009802-0140,380,0,0,0,0,0,1,0,0,0,0,1,660,0,4
2,009830-0020,360,0,0,0,0,0,1,0,0,0,0,1,1020,0,4
3,009830-0160,690,0,0,0,0,0,1,0,0,0,0,1,1000,0,4
4,010050-0180,60,0,0,0,0,0,1,0,0,0,0,1,440,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181505,197220-1902,0,0,0,0,0,0,1,0,0,0,0,1,240,0,2
181506,197220-2224,0,0,0,0,0,0,1,0,0,0,0,1,270,0,3
181507,197220-5172,0,0,0,0,0,0,1,0,0,0,0,1,290,0,3
181508,197220-5173,0,0,0,0,0,0,1,0,0,0,0,1,230,0,3


### Eliminate all sales not in 2019

In [16]:
res_sales.loc[res_sales['Parcel_ID']=='082211-9001']

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,Parcel_ID,SaleYear,SaleCount


In [17]:
df_resbldg.loc[(df_resbldg.Major=='082211')& (df_resbldg.Minor == '9001')].head()

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,DirectionSuffix,ZipCode,Stories,BldgGrade,BldgGradeVar,SqFt1stFloor,SqFtHalfFloor,SqFt2ndFloor,SqFtUpperFloor,SqFtUnfinFull,SqFtUnfinHalf,SqFtTotLiving,SqFtTotBasement,SqFtFinBasement,FinBasementGrade,SqFtGarageBasement,SqFtGarageAttached,DaylightBasement,SqFtOpenPorch,SqFtEnclosedPorch,SqFtDeck,HeatSystem,HeatSource,BrickStone,ViewUtilization,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
59918,082211,9001,20,1,20904 677TH PL SE 98045,20904,,,677TH,PL,SE,98045,1,3,0,340,0,0,0,0,0,340,0,0,0,0,0,,100,0,0,0,0,0,,0,0,0,0,1,0,0,0,1940,0,0,25,0,4,0
59919,082211,9001,1,1,20904 677TH PL SE 98045,20904,,,677TH,PL,SE,98045,1,3,0,430,0,0,0,0,0,430,0,0,0,0,0,,0,0,0,0,0,0,,0,0,0,0,1,0,0,0,1957,0,0,25,0,3,0
59920,082211,9001,14,1,20904 677TH PL SE 98045,20904,,,677TH,PL,SE,98045,1,4,0,320,0,0,0,0,0,320,0,0,0,0,0,,25,0,0,0,0,0,,0,0,0,0,0,0,0,0,1940,0,0,25,0,3,0
59921,082211,9001,17,1,20904 677TH PL SE 98045,20904,,,677TH,PL,SE,98045,1,5,0,430,0,0,0,0,0,430,0,0,0,0,0,,70,0,0,0,0,0,,0,0,0,0,1,0,0,0,1946,0,0,25,0,4,0
59922,082211,9001,5,1,20904 677TH PL SE 98045,20904,,,677TH,PL,SE,98045,1.5,5,0,540,270,0,0,0,0,810,0,0,0,0,0,,200,0,0,0,0,0,,0,0,0,0,0,0,0,0,1940,0,0,25,0,3,0
